In [1]:
# load credentials
import os
from dotenv import load_dotenv
load_dotenv()
HF_TOKEN = os.getenv("HUGGINGFACEHUB_API_TOKEN")

In [2]:
# load modules
from langchain import hub
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader

from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate

from langchain_huggingface import ChatHuggingFace, HuggingFaceEmbeddings, HuggingFacePipeline
from langchain_community.embeddings import HuggingFaceBgeEmbeddings

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [2]:
import os
from langchain_community.document_loaders import TextLoader

folder_path = "web_scraping"

# Get all .txt files in the folder and limit to first 100
txt_files = [file for file in os.listdir(folder_path) if file.endswith('.txt')][:50]


all_pages = []  # To store pages from all files

for txt in txt_files:
    # print(f"Processing {txt} file...")  # Optional: Show progress
    loader = TextLoader(file_path=os.path.join(folder_path, txt), encoding="utf-8")
    
    # Load the documents for this file and append to `all_pages`
    for doc in loader.lazy_load():
        all_pages.append(doc)

# Output all collected pages
print(f"✅ Total pages loaded: {len(all_pages)}")


✅ Total pages loaded: 50


In [4]:
# Splitting document
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap = 300,
        separators=[
        "\n"]
)

splits = text_splitter.split_documents(all_pages)
# splits

In [5]:
len(splits)

189

# Embedding model: BAAI/bge-m3

In [7]:
from langchain_huggingface import HuggingFaceEmbeddings

hf_embedding_model = HuggingFaceEmbeddings(
    model_name="BAAI/bge-m3",
    show_progress=True,
)

print("embedding model loaded")

c:\Users\ASUS\miniconda3\envs\torch\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


embedding model loaded


In [9]:
splits[20].page_content

'ঢাকার বিভিন্ন জায়গা হতে শাহবাগগামী অনেক বাস চলাচল করে। এসব বাসে কিংবা পছন্দের উপায়ে শাহবাগ এসে রিকশা বা পায়ে হেঁটে কলা ভবনের সামনে অপরাজেয় বাংলা দেখতে যেতে পারবেন।\nফিচার ইমেজ: Shiropa Shahreen\nভ্রমণ সংক্রান্ত যে কোন তথ্য ও আপডেট জানতে ফলো করুন আমাদের ফেসবুক পেইজ এবং জয়েন করুন আমাদের ফেসবুক গ্রুপে।\n© সর্বস্বত্ব সংরক্ষিত । এই ওয়েবসাইটের কোনো লেখা অনুমতি ছাড়া ব্যবহার বেআইনি।'

In [11]:
texts = []
for i in range(len(splits)//5):
    texts.append(splits[i].page_content)

embeddings = hf_embedding_model.embed_documents(texts)

Batches: 100%|██████████| 2/2 [00:27<00:00, 13.73s/it]


In [43]:
# embeddings[50]

In [13]:
from astrapy import DataAPIClient

ASTRA_DB_APPLICATION_TOKEN = os.getenv("ASTRA_DB_APPLICATION_TOKEN")
ASTRA_DB_API_ENDPOINT = os.getenv("ASTRA_DB_API_ENDPOINT")
# ASTRA_DB_KEYSPACE = os.getenv("ASTRA_DB_KEYSPACE")

# Initialize the client
client = DataAPIClient(ASTRA_DB_APPLICATION_TOKEN)
db = client.get_database_by_api_endpoint(
  ASTRA_DB_API_ENDPOINT
)

print(f"Connected Database: {db.info().name}\nCollections found: {db.list_collection_names()}")

Connected Database: research_paper_bot
Collections found: ['pregnancy_bot', 'travel_bot']


In [14]:
# create collection
from langchain_astradb import AstraDBVectorStore

vector_store = AstraDBVectorStore(
    collection_name="test_travel_kb",
    embedding=hf_embedding_model,
    api_endpoint=ASTRA_DB_API_ENDPOINT,
    token=ASTRA_DB_APPLICATION_TOKEN,
    namespace="default_keyspace",
    # autodetect_collection=True,   # set to True to use it
)

vector_store.collection_name

Batches: 100%|██████████| 1/1 [00:00<00:00,  2.74it/s]


'test_travel_kb'

In [49]:
type(splits)

list

In [15]:
for i in range(0, len(splits), 10):
    chunk = splits[i:i+10]
    try:
        # Add the chunk to the vector store
        vector_store.add_documents(documents=chunk)
        print(f"Chunk {i//10} added successfully")
    except Exception as e:
        print(f"Error adding chunk {i//10}: {e}")
        continue
    

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches: 100%|██████████| 1/1 [00:05<00:00,  5.94s/it]


Chunk 0 added successfully


Batches: 100%|██████████| 1/1 [00:05<00:00,  5.84s/it]


Chunk 1 added successfully


...

Chunk 16 added successfully


Batches: 100%|██████████| 1/1 [00:06<00:00,  6.59s/it]


Chunk 17 added successfully


Batches: 100%|██████████| 1/1 [00:05<00:00,  5.29s/it]


Chunk 18 added successfully


In [16]:
# as data is already loaded

vector_store = AstraDBVectorStore(
    embedding = hf_embedding_model,
    collection_name="test_travel_kb",
    api_endpoint=ASTRA_DB_API_ENDPOINT,
    token=ASTRA_DB_APPLICATION_TOKEN,
    namespace="default_keyspace",
    autodetect_collection=True,
)

## Vector Retriever

In [19]:
vec_retriever = vector_store.as_retriever(search_type="similarity", 
                                          search_kwargs={"k": 2})

In [20]:
# sanity check for similarity search
results = vector_store.similarity_search_with_score(
    "গাজীপুরে ঘুরে দেখার মত কি কি আছে?",
    k=2,
)
for res, score in results:
    print(f"*** [Similarity score={score:3f}]\n{res.page_content}\n[{res.metadata}]\n")

Batches: 100%|██████████| 1/1 [00:00<00:00,  4.81it/s]


*** [Similarity score=0.791149]
নিজস্ব বা গণপরিবহনে গাজীপুরের চৌরাস্তায় এসে সেখান থেকে গাজীপুর ডিসি অফিসের সামনে (রাজবাড়ী) যেতে হবে। ডিসি অফিসের সামনে থেকে সিএনজিতে শিমুলতলী রোড ধরে ৭ কিলোমিটার দূরত্বে অবস্থিত আরশিনগর হলিডে রিসোর্টে যেতে পারবেন।
পাজুলিয়া, জয়দেবপুর, গাজীপুর।মোবাইলঃ 01976462661, 01976462665, 01626462661ই-মেইলঃ info@arshinagarresort.com Website
গাজীপুরের সুন্দর আরও কিছু রিসোর্টঢাকার কাছে হওয়ায় গাজীপুর জেলায় আরও বেশ কিছু সুন্দর ও মনোরম রিসোর্ট গড়ে উঠেছে। তার মধ্যে উল্লেখযোগ্য গুলো হলো জল জঙ্গলের কাব্য, নক্ষত্র বাড়ি রিসোর্ট, ছুটি রিসোর্ট, অঙ্গনা রিসোর্ট ইত্যাদি।
ফিচার ইমেজ: দিপক কুমার
ভ্রমণ সংক্রান্ত যে কোন তথ্য ও আপডেট জানতে ফলো করুন আমাদের ফেসবুক পেইজ এবং জয়েন করুন আমাদের ফেসবুক গ্রুপে।
© সর্বস্বত্ব সংরক্ষিত । এই ওয়েবসাইটের কোনো লেখা অনুমতি ছাড়া ব্যবহার বেআইনি।
[{'source': 'web_scraping\\আরশিনগর হলিডে রিসোর্ট.txt'}]

*** [Similarity score=0.753260]
আদমজী সংলগ্ন এলাকায় সাথি হোটেল, রেসিপি চাইনিজ, প্রিতুলস ফুড জোন ও বিসমিল্লাহ্‌ বিরিয়ানি হাউজের মত বেশ কিছু খাবারের হেটেল 